In [2]:
from sqlalchemy import create_engine, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.orm import DeclarativeBase, mapped_column, relationship, sessionmaker
from datetime import datetime

## viena lentele - projektai

In [ ]:
db_engine = create_engine('sqlite:///projektai.db', echo=True)

In [ ]:
from typing import Any


class Base(DeclarativeBase):
    pass


class Projektas(Base):
    __tablename__ = "projektas"
    id = mapped_column(Integer, primary_key=True)
    name = mapped_column(String(50), nullable=False)
    price = mapped_column(Float(2), nullable=False)
    created = mapped_column(DateTime, default=datetime.utcnow)

    def __init__(self, **kw: Any):
        # super().__init__(**kw)
        for key, value in kw.items():
            setattr(self, key, value)
    
    def __repr__(self) -> str:
        return f"({self.id}, {self.name}, {self.price}, {self.created})"


In [ ]:
Base.metadata.create_all(db_engine)

In [ ]:
from sqlalchemy.orm import Session

In [ ]:
with Session(db_engine) as session:
    python_1 = Projektas(name="Python Kursai Pradedantiesiems", price=2000)
    session.add(python_1)
    session.commit()

In [ ]:
with Session(db_engine) as session:
    db_1 = Projektas(name="Duomenų Bazių su Python kursas", price=5000)
    frontas = Projektas(name="Frontendo kursai: HTML, CSS", price=1500)
    django = Projektas(name="Django kursas", price=7000)
    session.add_all([db_1, frontas, django])
    session.commit()

In [ ]:
with Session(db_engine) as session:
    projektai = session.query(Projektas).all()

In [ ]:
print(projektai)

In [ ]:
for projektas in projektai:
    print(projektas)

In [ ]:
with Session(db_engine) as session:
    django_kursas = session.query(Projektas).filter_by(name="Django kursas").one()
print(django_kursas)

In [ ]:
with Session(db_engine) as session:
    pigus = session.query(Projektas).filter(Projektas.price <= 3000)
    pigus = pigus.order_by(Projektas.price).all()
for projektas in pigus:
    print(projektas.price, projektas.name)

In [ ]:
with Session(db_engine) as session:
    kursai = session.query(Projektas)
    kursai = kursai.filter(Projektas.name.ilike("%Python%")).all()
for kursas in kursai:
    print(kursas)

In [ ]:
with Session(db_engine) as session:
    kursai = session.query(Projektas)
    kursai = kursai.filter(Projektas.name.ilike("%Python%")).all()
    kursai[0].price = 4000
    print(kursai[0])
    session.commit()

In [ ]:
with Session(db_engine) as session:
    print(projektas)
    projektas.price = 3000
    session.commit()

In [ ]:
from sqlalchemy.orm import sessionmaker
session = sessionmaker(bind=db_engine)()

projektas = session.query(Projektas).filter(Projektas.name.like("%Django%")).one()
print(projektas)

In [ ]:
projektas.price = 7775
session.commit()

## susijusios lenteles - tevai/vaikai

In [3]:
from typing import Any


class Base(DeclarativeBase):
    pass

In [ ]:
engine = create_engine('sqlite:///m21_seima.db')
session = sessionmaker(bind=engine)()

In [ ]:
class Tevas(Base):
    __tablename__ = "tevas"
    id = mapped_column(Integer, primary_key=True)
    vardas = mapped_column("vardas", String(50))
    pavarde = mapped_column("pavarde", String(50))
    vaikai = relationship("Vaikas", back_populates="tevas")


class Vaikas(Base):
    __tablename__ = "vaikas"
    id = mapped_column(Integer, primary_key=True)
    vardas = mapped_column("vardas", String(50))
    pavarde = mapped_column("pavarde", String(50))
    mokykla = mapped_column("mokykla", String(50))
    tevas_id = mapped_column(Integer, ForeignKey("tevas.id"))
    tevas = relationship("Tevas", back_populates="vaikai")


Base.metadata.create_all(engine)

In [ ]:
kestutis = Tevas(vardas="Kestutis", pavarde="J")
emilija = Vaikas(vardas="Emilija", pavarde="J", mokykla="gera", tevas=kestutis)
maja = Vaikas(vardas="Maja", pavarde="J")
marco = Vaikas(vardas="Marco", pavarde="J")
kestutis.vaikai.append(maja)
marco.tevas = kestutis
session.add_all([kestutis, emilija, maja, marco])
session.commit()

In [ ]:
tevas = session.query(Tevas).first()
print(tevas.vardas)

In [ ]:
for vaikas in tevas.vaikai:
    print(vaikas.vardas, vaikas.pavarde)

In [ ]:
tevas.vaikai[1].pavarde = "Ja"
session.commit()

In [ ]:
visi_vaikai = session.query(Vaikas).all()
for vaikas in visi_vaikai:
    print(vaikas.vardas, vaikas.tevas)

In [ ]:
tevas.vaikai.remove(visi_vaikai[1])
session.commit()

In [ ]:
visi_vaikai[0].tevas = None
session.commit()

## m2m

In [4]:
from sqlalchemy import Table, Column

engine = create_engine('sqlite:///m2m_seima.db')
session = sessionmaker(bind=engine)()

In [5]:
sasajos_lentele = Table('tevai_vaikai', Base.metadata, 
    Column('id', Integer, primary_key=True),
    Column('tevas_id', Integer, ForeignKey('tevas.id')),
    Column('vaikas_id', Integer, ForeignKey('vaikas.id')),
)

class Tevas(Base):
    __tablename__ = "tevas"
    id = mapped_column(Integer, primary_key=True)
    vardas = mapped_column("vardas", String(50))
    pavarde = mapped_column("pavarde", String(50))
    vaikai = relationship(
        "Vaikas", 
        secondary=sasajos_lentele, 
        back_populates='tevai',
    )

class Vaikas(Base):
    __tablename__ = "vaikas"
    id = mapped_column(Integer, primary_key=True)
    vardas = mapped_column("vardas", String(50))
    pavarde = mapped_column("pavarde", String(50))
    tevai = relationship(
        "Tevas",
        secondary=sasajos_lentele,
        back_populates='vaikai',
    )

Base.metadata.create_all(engine)

In [8]:
kestutis = Tevas(vardas="Kestutis", pavarde="J")
nicoletta = Tevas(vardas="Nicoletta", pavarde="P")
emilija = Vaikas(vardas="Emilija", pavarde="J")
maja = Vaikas(vardas="Maja", pavarde="Ja")
marco = Vaikas(vardas="Marco", pavarde="Jo")
kestutis.vaikai.append(emilija)
kestutis.vaikai.append(marco)
kestutis.vaikai.append(maja)
emilija.tevai.append(nicoletta)
maja.tevai.append(nicoletta)
marco.tevai.append(nicoletta)
session.add_all([kestutis, emilija, maja, marco, nicoletta])
session.commit()

In [10]:
tevas = session.query(Tevas).filter(Tevas.vardas == "Kestutis").first()
mama = session.query(Tevas).filter(Tevas.vardas == "Nicoletta").first()
for vaikas in tevas.vaikai:
    print(vaikas.vardas, vaikas.pavarde)
for vaikas in mama.vaikai:
    print(vaikas.vardas, vaikas.pavarde)

Emilija J
Marco Jo
Maja Ja
Emilija J
Maja Ja
Marco Jo


In [11]:
vaikai = session.query(Vaikas).all()
for vaikas in vaikai:
    print(vaikas.vardas, vaikas.pavarde)
    for tevas in vaikas.tevai:
        print("--", tevas.vardas, tevas.pavarde)

Emilija J
-- Nicoletta P
-- Kestutis J
Marco Jo
-- Nicoletta P
-- Kestutis J
Maja Ja
-- Nicoletta P
-- Kestutis J


In [12]:
mama.vaikai[0].tevai[1].vardas

'Kestutis'